# 🥱 LazyMergekit + 🤖 Phi3 = ❤️

![](https://ebqbyiy3cbe.exactdn.com/wp-content/uploads/2024/04/423caff5-d0f3-420b-8c66-108aeba5e2d0.jpg?strip=all&lossy=1&quality=88&sharp=1&ssl=1)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

♻️ Adapted by [@paulilioaica](https://huggingface.co/paulilioaica) to be able to create Mixture of Experts with Phi3s

This notebook allows you to easily create mixture of experts using multiple Phi3s using [mergekit](https://github.com/paulilioaica/mergekit-phi3).

*Special thanks to [@maximelabonne](https://github.com/cg123) for the Phixtral initiative.

In [ ]:
out_path = "phi3s"

MODEL_NAME =  "Phi3Mix"

yaml_config = f"""
base_model: microsoft/Phi-3-mini-4k-instruct
gate_mode: cheap_embed
experts_per_token: 1
dtype: float16
experts:
  - source_model: microsoft/Phi-3-mini-4k-instruct
    positive_prompts: ["research, logic, math, science"]
  - source_model: microsoft/Phi-3-mini-4k-instruct
    positive_prompts: ["creative, art"]
"""


# Save config as yaml file
with open('config.yaml', 'w', encoding="utf-8") as f:
    f.write(yaml_config)

In [ ]:
# @title ## Install dependencies

# @markdown ### Install dependencies from git
# @markdown Select your runtime (CPU, High RAM, GPU)

runtime = "CPU" # @param ["CPU", "CPU + High-RAM", "GPU"]


branch = "mixtral"
trust_remote_code = True
!git clone -b mixtral https://github.com/paulilioaica/mergekit-phi3
!cd mergekit-phi3 && pip install -qqq -e . --progress-bar off
!pip install -qqq -U transformers --progress-bar off


In [ ]:
# @title ## Building the CLI

# @markdown ### Building the CLI


cli = f"mergekit-moe config.yaml {out_path}  --allow-crimes --trust-remote-code --out-shard-size 5B --lazy-unpickle --copy-tokenizer --i-understand-this-is-not-useful-without-training"

# Additional arguments
if runtime == "GPU":
  cli += " --device cuda --low-cpu-memory"


In [ ]:
# @title ## Run merge!
# @markdown ### Start the merging process
!{cli}

In [ ]:
# @title ## Copy remaining Phi3 dependencies to output folder


!cp mergekit-phi3/mergekit/configuration_phi3.py {out_path}/configuration_phi3.py
!cp mergekit-phi3/mergekit/modeling_phi3.py {out_path}/modeling_phi3.py

In [ ]:
# @title ## Upload model to Hugging Face { display-mode: "form" }
!pip install -qU huggingface_hub

username = 'YOUR_USERNAME'
token = 'YOUR_TOKEN'
license = "apache-2.0"


import yaml

from huggingface_hub import ModelCard, ModelCardData, HfApi
from jinja2 import Template

template_text = """
---
license: {{ license }}
base_model:
{%- for model in models %}
  - {{ model }}
{%- endfor %}
tags:
- moe
- merge
- mergekit
- lazymergekit
- phi3_mergekit

{%- for model in models %}
- {{ model }}
{%- endfor %}
---

# {{ model_name }}

{{ model_name }} is a Mixture of Experts (MoE) made with the following models using [Phi3_LazyMergekit](https://colab.research.google.com/drive/1Upb8JOAS3-K-iemblew34p9h1H6wtCeU?usp=sharing):

{%- for model in models %}
* [{{ model }}](https://huggingface.co/{{ model }})
{%- endfor %}


## 🧩 Configuration

```yaml
{{- yaml_config -}}
```

## 💻 Usage

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model = "{{ username }}/{{ model_name }}"

tokenizer = AutoTokenizer.from_pretrained(model)

model = AutoModelForCausalLM.from_pretrained(
    model,
    trust_remote_code=True,
)

prompt="How many continents are there?"
input = f"<|system|>You are a helpful AI assistant.<|end|><|user|>{prompt}<|assistant|>"
tokenized_input = tokenizer.encode(input, return_tensors="pt")

outputs = model.generate(tokenized_input, max_new_tokens=128, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
print(tokenizer.decode(outputs[0]))
```
"""

# Create a Jinja template object
jinja_template = Template(template_text.strip())

# Fill the template
data = yaml.safe_load(yaml_config)
models = [model['source_model'] for model in data['experts']]

content = jinja_template.render(
    model_name=MODEL_NAME,
    models=models,
    yaml_config=yaml_config,
    username=username,
    license=license
)

# Save the model card
card = ModelCard(content)
card.save(f'{out_path}/README.md')

# Defined in the secrets tab in Google Colab
api = HfApi(token=token)

# Upload merge folder
api.create_repo(
    repo_id=f"{username}/{MODEL_NAME}",
    repo_type="model",
    exist_ok=True,
)
api.upload_folder(
    repo_id=f"{username}/{MODEL_NAME}",
    folder_path=out_path,
)